In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [2]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

words = ['E_no']
nwords = ['__', 'all']

for i in range(len(files)):
    if all((s in files[i]) for s in words):
        if all((s not in files[i]) for s in nwords):
#     if 'E_no' in files[i]:
#         if '__' not in files[i]:
            filenames.append(files[i])
            print(i, files[i])

2 Ag_100_u2_RPBE_no001_CO_n1_d9.traj
6 Ag_100_u2_RPBE_no002_CO_n1_d9.traj
10 Ag_100_u2_RPBE_no002_CO_n2_d3.traj
14 Ag_100_u2_RPBE_no003_CO_n1_d9.traj
18 Ag_100_u2_RPBE_no003_CO_n3_d3.traj
22 Ag_100_u2_RPBE_no004_CO_n4_d3.traj
26 Ag_100_u2_RPBE_no005_CO_n3_d3.traj
30 Ag_100_u2_RPBE_no006_CO_n4_d3.traj
34 Ag_100_u2_RPBE_no007_CO_n2_d3.traj
38 Ag_100_u2_RPBE_no008_CO_n2_d3.traj
42 Ag_100_u2_RPBE_no009_CO_n3_d3.traj
46 Ag_100_u2_RPBE_no011_CO_n2_d3.traj
50 Ag_100_u2_RPBE_no012_CO_n3_d3.traj
54 Ag_100_u2_RPBE_no013_CO_n4_d3.traj
58 Ag_100_u2_RPBE_no014_CO_n2_d3.traj
62 Ag_100_u2_RPBE_no015_CO_n2_d3.traj
68 Ag_110_u2_RPBE_no001_CO_n1_d9.traj
72 Ag_110_u2_RPBE_no002_CO_n1_d9.traj
76 Ag_110_u2_RPBE_no002_CO_n2_d6.traj
80 Ag_110_u2_RPBE_no003_CO_n1_d9.traj
84 Ag_110_u2_RPBE_no003_CO_n3_d3.traj
88 Ag_110_u2_RPBE_no004_CO_n1_d9.traj
92 Ag_110_u2_RPBE_no005_CO_n3_d3.traj
96 Ag_110_u2_RPBE_no006_CO_n4_d3.traj
100 Ag_110_u2_RPBE_no007_CO_n2_d6.traj
104 Ag_110_u2_RPBE_no008_CO_n3_d3.traj
108 Ag_110_u

In [3]:
# ### DELETE ###
# for file in filenames:
#     try:
#         A = make_database(file)
#         A.delete_from_database()
#     except:
#         print(file, 'Some error occured.')

In [4]:
### ADD and UPDATE ENERGY ###
for file in filenames:
#     try:
    A = make_database(file, 'adsE_method2_new')
    A.add_to_database()
    A.update_Energy()
    A.update_adsorbates_correlation(expression=2, force_update=False)
#     except:
#         print(file, 'Some error occured.')

-----------------------------------------------------------
Ag_100_u2_RPBE_no001_CO_n1_d9.traj
Not in database.
One post: 5eaddde61515f3c56e073875

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlation updated.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n1_d9.traj
Not in database.
One post: 5eadddf31515f3c56e073877

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlation updated.
-----------------------------------------------------------
Ag_100_u2_RPBE_no002_CO_n2_d3.traj
Not in database.
One post: 5eadde001515f3c56e073879

E_each_ads and E_residue/suratom updated.
Adsorbate correlation updated.
-----------------------------------------------------------
Ag_100_u2_RPBE_no003_CO_n1_d9.traj
Not in database.
One post: 5eadde0d1515f3c56e07387b

Updating energy is for surface with more than 2 adsorbates.
Adsorbate correlation updated.
-----------------------------------------------------------
Ag

One post: 5eaddf2a1515f3c56e0738b3

Reference data is invalid. Each adsorption energy cannot be calculated.
Could not get Each adsorbates energy.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/katsuyut/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-8760f9dad18f>", line 7, in <module>
    A.update_adsorbates_correlation(expression=2, force_update=False)
  File "/home/katsuyut/research/coverage-effect/modules/MAPost.py", line 468, in update_adsorbates_correlation
    b_mat, nads = get_coordination_matrix(rratoms, expression)
  File "/home/katsuyut/research/coverage-effect/modules/MAPost.py", line 192, in get_coordination_matrix
    coordination, cindexes = __get_coordination_string_mod(nn_info)
  File "/home/katsuyut/research/coverage-effect/modules/MAPost.py", line 121, in __get_coordination_string_mod
    for neighbor_info in nn_info
  File "/home/katsuyut/research/coverage-effect/modules/MAPost.py", line 122, in <listcomp>
    if neighbor_info['site'].species_string != 'Kr']
  File "/home/katsuyut/.loca

KeyboardInterrupt: 

In [6]:
file = 'Cu_100_u2_RPBE_no012_CO_n3_d3.traj'
A = make_database(file)
A.check_database()

-----------------------------------------------------------
Cu_100_u2_RPBE_no012_CO_n3_d3.traj


{'_id': ObjectId('5e7e087c3c2b2ac982d21472'),
 'name': 'Cu_100_u2_RPBE_no012_CO_n3_d3.traj',
 'isvalid': 'yes',
 'ispredictable': 'yes',
 'element': 'Cu',
 'face': '100',
 'unitlength': 2,
 'xc': 'RPBE',
 'adsorbate': 'CO',
 'numberofads': 3,
 'coverage': 0.75,
 'surfatomnum': 4,
 'E': -91.5562245,
 'bareE': -47.97462411,
 'E_ads': -14.48543631,
 'totaladsE': -0.12529145999998548,
 'aveadsE/suratom': -0.03132286499999637,
 'aveadsE/ads': -0.04176381999999516,
 'E_int_space': 1.422352740000008,
 'sumE_each_ads': -1.3633090699999784,
 'E_residue/suratom': -0.04608378250000378,
 'area': 27.043730169821497,
 'density': 0.11093144256215605,
 'igroups': [2, 2, 1],
 'rgroups': [2, 2, 1],
 'converged': 'yes',
 'is_adsorbed': 'yes',
 'kept_sites': 'yes',
 'E_not_exceeded': 'yes',
 'minimum_distance': 2,
 'ads_dist2': 8.0,
 'ads_dist3': 28.0}

In [18]:
query('Zn_001_u2_RPBE_no003_CO_n1_d8.traj','local')

Atoms(symbols='Zn16CO', pbc=True, cell=[[5.323107383026382, 0.0, 3e-16], [-2.661553691513193, 4.609946220773349, 3e-16], [0.0, 0.0, 37.388231425177516]], constraint=FixAtoms(indices=[0, 1, 4, 5, 8, 9, 12, 13]), calculator=SinglePointCalculator(...))